# Luna Quantum SDK: Traveling Salesman Problem Demo

Welcome to the **Luna Quantum SDK** demonstration! This comprehensive notebook showcases how Luna Quantum makes quantum optimization accessible, powerful, and easy to use through a complete Traveling Salesman Problem (TSP) solution.

## 🚀 Why Luna Quantum?

Luna Quantum is a cutting-edge SDK that bridges the gap between classical and quantum computing, offering:

- **Unified Programming Model**: Write once, run on classical CPUs, quantum simulators, or real quantum hardware
- **Smart Abstractions**: Focus on problem modeling while Luna handles algorithm-specific transformations
- **Plug-and-Play Architecture**: Seamlessly switch between algorithms and backends without code changes
- **Production-Ready**: Enterprise-grade quantum cloud platform with secure token management

## 🎯 What this Notebook will Show

This demo highlights Luna Quantum's key capabilities through a practical TSP implementation:

1. **Rapid Prototyping**: Use predefined optimization patterns for quick problem setup
2. **Advanced Modeling**: Build custom models with AqModels for complete control
3. **Hybrid Solving**: Compare classical and quantum algorithms on the same problem
4. **Seamless Deployment**: Run on classical machines, cloud simulators, or quantum hardware

## 📍 The Challenge

We'll solve the TSP for 4 German cities: **Berlin**, **Hamburg**, **Munich**, and **Bonn**. The goal is to find the shortest route that visits all cities exactly once and returns to the starting point—a perfect demonstration of Luna Quantum's optimization capabilities.

## 📋 Prerequisites

Before running this notebook, ensure you have:
- ✅ Luna Quantum SDK installed (`pip install luna-quantum`)
- ✅ D-Wave token set as environment variable: `DWAVE_TOKEN`
- ✅ All dependencies installed (see `pyproject.toml`)

**Need help with setup?** Check the project README for detailed instructions.

## 1. 🔧 Setup and Luna Quantum SDK Integration

Luna Quantum's elegant API design makes getting started incredibly simple. With just a few imports, you gain access to a comprehensive suite of optimization tools.

### 🧠 Core Luna Quantum Components:

#### **Modeling Framework**
- **`Model`**: Universal modeling interface that works across classical and quantum algorithms
- **`Variable`**: Flexible variable types with automatic constraint handling
- **`Vtype`**: Variable type definitions (Binary, Integer, Continuous)

#### **Orchestration & Execution**
- **`LunaSolve`**: Main orchestration class for job management and execution
- **`algorithms`** & **`backends`**: Plug-and-play algorithm and execution environment selection
- **`LpTranslator`**: Seamless conversion between different model formats

#### **Pre-built Solutions**
- **`TravellingSalesmanProblem`**: Ready-to-use optimization patterns for rapid development

#### **Utility Functions**
- **`plot_graph_on_map`**: Interactive geographic visualization
- **`plot_solution_tour`**: Solution visualization on maps
- **`calculate_distance_matrix`**: Haversine distance calculations

### 🚀 The Luna Quantum Workflow

Luna Quantum follows a simple 6-step process:

![LunaSolve Flow](plots/LunaSolveFlow.png)


In [29]:
from luna_quantum import Model,Variable, Vtype
from luna_quantum.translator import LpTranslator
import networkx as nx
import itertools
import os
from luna_quantum.solve.use_cases import TravellingSalesmanProblem
from luna_quantum import LunaSolve
from luna_quantum.solve import algorithms, backends
from luna_quantum.client.schemas import QpuToken
import getpass

# Use Case utils
from utils.plotting import plot_graph_on_map, plot_solution_tour
from utils.traveling_sales_man import calculate_distance_matrix, extract_tour_from_solution

if "LUNA_API_KEY" not in os.environ:
    # Prompt securely for the key if not already set
    os.environ["LUNA_API_KEY"] = getpass.getpass("Enter your Luna API key: ")


# Complete Luna Flow
![Alt text](plots/FlowComplete.png)


### 🚀 Quick Start Example

Before diving into detailed modeling, let's see Luna Quantum in action with a pre-built model:

In [32]:
# 1. Load a pre-built model
m = LpTranslator.to_aq('tsp_model') # Note: We'll build this model in the sections below

# 2. Select Algorithm
sim_annealer = algorithms.SimulatedAnnealing(num_reads=100)

# 3. / 4. / 5.  Compute optimization
job = sim_annealer.run(model=m)

# 6. Analyze solution
solution = job.result()
print(solution)

Output()

Output()

2025-06-17 14:58:43 INFO     {"model_id":"6851310efeb4512569869b57","solver_name":"SA","provider":"dwave","paramete
                             rs":{"num_reads":100,"num_sweeps":1000,"beta_range":null,"beta_schedule_type":"geometr
                             ic","initial_states_generator":"random","num_sweeps_per_beta":1,"seed":null,"beta_sche
                             dule":null,"initial_states":null,"randomize_order":false,"proposal_acceptance_criteria
                             ":"Metropolis"},"name":null}

Output()

Output()

x_Ham x_Ham x_Ham x_Mun x_Mun x_Mun x_Bon x_Bon x_Bon │ feas   raw   obj count
    1     0     0     0     1     0     0     0     1 │    t 1.8e3 1.8e3    18
    0     0     1     0     1     0     1     0     0 │    t 1.8e3 1.8e3    17
    0     1     0     0     0     1     1     0     0 │    t 2.0e3 2.0e3    10
    0     1     0     1     0     0     0     0     1 │    t 2.0e3 2.0e3    17
    1     0     0     0     0     1     0     1     0 │    t 1.6e3 1.6e3    19
    0     0     1     1     0     0     0     1     0 │    t 1.6e3 1.6e3    19

Total samples: 6
Total variables: 9

Timing:
Total: 0.019366026s


**⚠️ Note**:
The above is a preview of the complete workflow. We'll build the actual model step-by-step in the following sections.

## 2. 📊 Problem Data Setup

### 🌍 Real-World Optimization Challenge

Our TSP demonstration uses 4 major German cities with their precise GPS coordinates:

| City | Coordinates | Population | Significance |
|------|-------------|------------|--------------|
| **Berlin** | 52.52°N, 13.405°E | 3.7M | Germany's capital and largest city |
| **Hamburg** | 53.55°N, 9.99°E | 1.9M | Major port city in northern Germany |
| **Munich** | 48.13°N, 11.58°E | 1.5M | Bavaria's capital in southern Germany |
| **Bonn** | 50.74°N, 7.10°E | 330K | Former capital of West Germany |

### 🔬 Distance Calculation Methods

The `calculate_distance_matrix()` function demonstrates Luna Quantum's preprocessing capabilities:

- **Haversine Formula**: Calculates great-circle distances between GPS coordinates
- **Symmetric Matrix**: Ensures distance(A→B) = distance(B→A) for optimization efficiency
- **Real-World Accuracy**: Accounts for Earth's curvature, not simple Euclidean distance
- **Automatic Preprocessing**: Generates the cost matrix needed for TSP optimization

### 📏 Expected Distances (km)

The distance matrix shows realistic inter-city distances:
- Berlin ↔ Hamburg: ~255 km
- Berlin ↔ Munich: ~504 km  
- Berlin ↔ Bonn: ~478 km
- Hamburg ↔ Munich: ~612 km
- Hamburg ↔ Bonn: ~370 km
- Munich ↔ Bonn: ~434 km

In [18]:
cities = {
    "Berlin": (52.52, 13.405),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.582),
    "Bonn": (50.7374, 7.0982),
}
distance_matrix = calculate_distance_matrix(cities)
distance_matrix

,Berlin,Hamburg,Munich,Bonn
Berlin,0,255,504,478
Hamburg,255,0,612,370
Munich,504,612,0,434
Bonn,478,370,434,0


### 🔗 Graph Representation

We use NetworkX to create a mathematical representation of our TSP problem:

**Graph Structure:**
- **Nodes**: Cities with GPS coordinates as attributes
- **Edges**: All possible routes between cities with distance weights
- **Complete Graph**: Every city connects to every other city (K₄ graph)

In [19]:
# Create a graph
G = nx.Graph()

# Add nodes with their geographic coordinates
for city, (lat, lon) in cities.items():
    G.add_node(city, pos=(lon, lat))  # Note: NetworkX expects (x, y) = (lon, lat)

# Add edges
for city1, city2 in itertools.combinations(cities, 2):
    G.add_edge(city1, city2, weight=distance_matrix[city1][city2])

### Problem Visualization

Luna Quantum's visualization capabilities provide immediate problem insights and solution validation:

#### Interactive Map Features:
- **🔴 Red Markers**: Precise city locations with real geographic coordinates
- **🔵 Blue Connections**: All possible routes between cities with distance weighting
- **🗺️ Geographic Context**: Real-world spatial relationships on an interactive map


In [20]:
map = plot_graph_on_map(G = G, cities = cities)
map

# 🚀 Building Optimization Models with Luna Quantum

![LunaSolve Flow Model](plots/FlowModel.png)

## Method 1: Predefined Use Cases - Luna Quantum's Rapid Development Engine

Luna Quantum's predefined use cases represent a revolutionary approach to optimization development. Instead of reinventing the wheel for common problems, Luna Quantum provides battle-tested, optimized implementations that work out-of-the-box.

### Luna Quantum's Use Case Advantages:

🎯 **Zero Setup Time**: Transform raw data into optimized models in seconds  
🧠 **Expert Formulations**: Each use case embodies years of optimization research and best practices  
🔄 **Automatic Preprocessing**: Intelligent data validation, scaling, and constraint generation  
⚡ **Performance Optimized**: Pre-tuned for maximum efficiency across different algorithm types  
🛡️ **Production Ready**: Enterprise-grade reliability with comprehensive error handling  


In [21]:
# Instantiate Luna
ls = LunaSolve()
tsp_graph = nx.to_dict_of_dicts(G)

# Use predefined use case to define model
tsp = TravellingSalesmanProblem(graph=tsp_graph)
model = ls.model.create_from_use_case(name="predefined TSP", use_case=tsp)
print(ls.model.get_model(model.id))

Model: predefined TSP
Minimize
  1226 * x_0 * x_1 + 1226 * x_0 * x_2 + 1226 * x_0 * x_3 + 1226 * x_0 * x_4 
  + 255 * x_0 * x_5 + 255 * x_0 * x_7 + 1226 * x_0 * x_8 + 504 * x_0 * x_9 
  + 504 * x_0 * x_11 + 1226 * x_0 * x_12 + 478 * x_0 * x_13 + 478 * x_0 * x_15 
  + 1226 * x_1 * x_2 + 1226 * x_1 * x_3 + 255 * x_1 * x_4 + 1226 * x_1 * x_5 
  + 255 * x_1 * x_6 + 504 * x_1 * x_8 + 1226 * x_1 * x_9 + 504 * x_1 * x_10 
  + 478 * x_1 * x_12 + 1226 * x_1 * x_13 + 478 * x_1 * x_14 + 1226 * x_2 * x_3 
  + 255 * x_2 * x_5 + 1226 * x_2 * x_6 + 255 * x_2 * x_7 + 504 * x_2 * x_9 
  + 1226 * x_2 * x_10 + 504 * x_2 * x_11 + 478 * x_2 * x_13 + 1226 * x_2 * x_14 
  + 478 * x_2 * x_15 + 255 * x_3 * x_4 + 255 * x_3 * x_6 + 1226 * x_3 * x_7 
  + 504 * x_3 * x_8 + 504 * x_3 * x_10 + 1226 * x_3 * x_11 + 478 * x_3 * x_12 
  + 478 * x_3 * x_14 + 1226 * x_3 * x_15 + 1226 * x_4 * x_5 + 1226 * x_4 * x_6 
  + 1226 * x_4 * x_7 + 1226 * x_4 * x_8 + 612 * x_4 * x_9 + 612 * x_4 * x_11 
  + 1226 * x_4 * x_12 + 370 * 

## Method 2: AqModels - Luna Quantum's Advanced Custom Modeling Engine

**AqModels** represents Luna Quantum's core package for advanced optimization modeling.
This powerful framework combines the best practices of classical optimization with cutting-edge quantum capabilities, providing flexibility and control.

### 🌟 AqModels Revolutionary Features:

#### **Unified Classical-Quantum Interface**
- **Single API** for both classical and quantum algorithm development
- **Automatic problem transformations** (LP → QUBO, QUBO → Ising, etc.)
- **Algorithm-agnostic modeling** - write once, solve anywhere

#### **Intelligent Abstraction Layer**
- **Smart constraint handling** with automatic feasibility checking
- **Built-in variable management** with type safety and bounds checking  
- **Optimization-aware preprocessing** for maximum solver performance

#### **Production-Grade Reliability**
- **Comprehensive error checking** and validation
- **Scalable architecture** handling problems from toy examples to enterprise scale
- **Memory-efficient representations** for large-scale optimization

### 🚀 Position-Based TSP Formulation

The custom model below demonstrates AqModels' sophistication by implementing a **position-based TSP formulation**:

**Key Concepts:**
- **Fixed Starting City**: Berlin is always position 0 (reduces problem complexity)
- **Position Variables**: `x[city, position] = 1` if city is visited at that position
- **Constraint Types**: Each city visited once, each position filled once
- **Penalty vs Constraints**: Choose between hard constraints or penalty-based QUBO formulation

**Mathematical Formulation:**
- **Variables**: `x[i,j] ∈ {0,1}` where i=city, j=position
- **Objective**: Minimize total distance including start/end connections
- **Constraints**: Assignment constraints ensure valid tours

In [31]:
# Traveling Salesman Problem (TSP) - Position-Based Formulation
# This implementation uses a position-based approach where each city is assigned
# to a specific position in the tour (except the starting city which is fixed)

# Initialize the model and fix starting city
travel_cities = list(cities.keys())
start_city = travel_cities.pop(0)
n_nodes = len(cities)

model = Model()
x = {}
with model.environment:
    # Decision Variables x[i, j] = 1 if city i is visited at position j in the tour, 0 otherwise
    for i in travel_cities:
        for j in range(1, n_nodes):
            x[i, j] = Variable(f"x_{i}_{j}", vtype=Vtype.Binary)

# Objective Function: Minimize total travel distance

# Distance from and to starting city
for i in travel_cities:
    model.objective += distance_matrix[start_city][i] * x[i, 1]
    model.objective += distance_matrix[i][start_city] * x[i, n_nodes-1]

# Distance between consecutive positions in the tour
for pos in range(1, n_nodes-1):
    for i, j in itertools.combinations(travel_cities, 2):
        # If city i is at position 'pos' AND city j is at position 'pos+1'
        # then add distance from i to j and its reverse
        model.objective += distance_matrix[i][j] * x[i, pos] * x[j, pos+1]
        model.objective += distance_matrix[j][i] * x[j, pos] * x[i, pos+1]

# Constraints as penalty terms in objective
penalty_term = float(distance_matrix.max().max()) *10
constrained_model = False
from luna_quantum import quicksum
# Constraint 1: Each city (except the starting city) must be visited exactly once
for i in travel_cities:
    lhs = quicksum(x[i, j] for j in range(1, n_nodes))
    if constrained_model:
        model.add_constraint(lhs == 1, name=f"city_{i}_visited_once")
    else:
        model.objective += penalty_term * (lhs**2) + penalty_term - 2*lhs*penalty_term

# Constraint 2: # Each position (except position 0 which is fixed to start_city)
# must be occupied by exactly one city
for j in range(1, n_nodes):
    lhs = sum(x[i, j] for i in travel_cities)
    if constrained_model:
        model.add_constraint(lhs == 1, name=f"pos_{j}_occupied_once")
    else:
        model.objective += penalty_term * (lhs**2) + penalty_term - 2*lhs*penalty_term

# The final model
print(model)

# easily save model
LpTranslator.from_aq(model = model, filepath ='tsp_model')

Model: unnamed
Minimize
  12240 * x_Hamburg_1 * x_Hamburg_2 + 12240 * x_Hamburg_1 * x_Hamburg_3 
  + 12240 * x_Hamburg_1 * x_Munich_1 + 612 * x_Hamburg_1 * x_Munich_2 
  + 12240 * x_Hamburg_1 * x_Bonn_1 + 370 * x_Hamburg_1 * x_Bonn_2 
  + 12240 * x_Hamburg_2 * x_Hamburg_3 + 612 * x_Hamburg_2 * x_Munich_1 
  + 12240 * x_Hamburg_2 * x_Munich_2 + 612 * x_Hamburg_2 * x_Munich_3 
  + 370 * x_Hamburg_2 * x_Bonn_1 + 12240 * x_Hamburg_2 * x_Bonn_2 
  + 370 * x_Hamburg_2 * x_Bonn_3 + 612 * x_Hamburg_3 * x_Munich_2 
  + 12240 * x_Hamburg_3 * x_Munich_3 + 370 * x_Hamburg_3 * x_Bonn_2 
  + 12240 * x_Hamburg_3 * x_Bonn_3 + 12240 * x_Munich_1 * x_Munich_2 
  + 12240 * x_Munich_1 * x_Munich_3 + 12240 * x_Munich_1 * x_Bonn_1 
  + 434 * x_Munich_1 * x_Bonn_2 + 12240 * x_Munich_2 * x_Munich_3 
  + 434 * x_Munich_2 * x_Bonn_1 + 12240 * x_Munich_2 * x_Bonn_2 
  + 434 * x_Munich_2 * x_Bonn_3 + 434 * x_Munich_3 * x_Bonn_2 
  + 12240 * x_Munich_3 * x_Bonn_3 + 12240 * x_Bonn_1 * x_Bonn_2 
  + 12240 * x_Bonn_1

### 🔐 Authentication Setup

Before running optimization jobs, we need to set up authentication for quantum hardware access:

In [23]:
# Set up D-Wave quantum computing token

personal_qpu_token = ls.qpu_token.create(
    provider='dwave',
    name='my-dwave-token',
    token=os.environ.get("DWAVE_TOKEN"),  # Set this environment variable
    token_type="personal"
)


2025-06-17 14:51:42 ERROR    The Luna-Server reported the error 'QPU_TOKEN_NAME_ALREADY_EXISTS' with the message:  
                              The provided QPU-Token-Name already exists.

LunaServerError: The Luna-Server reported the error 'QPU_TOKEN_NAME_ALREADY_EXISTS' with the message:
 The provided QPU-Token-Name already exists.

**⚠️ Important Setup Notes:**
- Set your D-Wave token: `export DWAVE_TOKEN="your_token_here"`
- Get tokens from [D-Wave Leap](https://cloud.dwavesys.com/leap/)
- Tokens are securely managed by Luna Quantum
- No credentials stored in code - uses environment variables

### 🚀 Seamless Multi-Algorithm Execution

Luna Quantum's execution model demonstrates the power of our plug-and-play architecture. Below, we solve the **exact same model** using two completely different approaches:

1. **Classical Simulated Annealing**: Fast, reliable, runs on any hardware
2. **Quantum Annealing**: Leveraging quantum effects for potential speedups

#### **The Luna Quantum Promise:**
- **Identical code structure** regardless of algorithm choice
- **Automatic problem translation** to algorithm-specific formats
- **Unified result handling** across all execution methods
- **Transparent job management** with progress tracking and error handling

This flexibility means you can:
- **Start development** with classical algorithms on your laptop
- **Scale testing** using cloud simulators  
- **Deploy production** on quantum hardware
- **Compare performance** across multiple approaches

All while maintaining the same codebase and workflow!

![LunaSolve Flow Algorithm Selection](plots/FlowAlgorithm.png)

### 🔧 Backend and Algorithm Configuration

Luna Quantum's flexible architecture allows you to choose different execution environments:

In [24]:
# Classical simulation backend (no tokens required)
classic_backend = backends.DWave()

# Quantum hardware backend (requires D-Wave token)
from luna_quantum.client.schemas.qpu_token.qpu_token import PersonalQpuToken
personal_qpu_token = PersonalQpuToken(name='my-dwave-token')
qpu_backend = backends.DWaveQpu(token=personal_qpu_token)

# Algorithm selection with different backends
tabu_search = algorithms.TabuSearch(num_reads=100, backend=classic_backend)
quantum_annealing = algorithms.QuantumAnnealing(num_reads=100, backend=qpu_backend)

**Algorithm Comparison:**

- **Tabu Search**: Classical heuristic, fast execution, good for development
- **Quantum Annealing**: Quantum algorithm, potential for better solutions, requires quantum hardware

# Runninc model

![LunaSolve Flow Compute](plots/FlowCompute.png)

### 🚀 Parallel Job Execution

Luna Quantum supports running multiple optimization jobs concurrently:

In [25]:
# Submit both jobs simultaneously
print("🔄 Submitting optimization jobs...")
print("   → Classical Tabu Search")
print("   → Quantum Annealing (if token available)")

tabu_job = tabu_search.run(model)
quantum_job = quantum_annealing.run(model)

print("✅ Jobs submitted successfully!")
print("   → Job 1 ID:", tabu_job.id if hasattr(tabu_job, 'id') else 'Classical job')
print("   → Job 2 ID:", quantum_job.id if hasattr(quantum_job, 'id') else 'Quantum job')

🔄 Submitting optimization jobs...
   → Classical Tabu Search
   → Quantum Annealing (if token available)


Output()

Output()

2025-06-17 14:52:18 INFO     {"model_id":"6851310efeb4512569869b57","solver_name":"TS","provider":"dwave","paramete
                             rs":{"num_reads":100,"tenure":null,"timeout":100,"seed":null,"num_restarts":1000000,"e
                             nergy_threshold":null,"coefficient_z_first":null,"coefficient_z_restart":null,"lower_b
                             ound_z":null,"initial_states":null,"initial_states_generator":"random"},"name":null}

Output()

Output()

2025-06-17 14:52:20 INFO     {"model_id":"6851310efeb4512569869b57","solver_name":"QA","provider":"dwave","paramete
                             rs":{"anneal_offsets":null,"anneal_schedule":null,"annealing_time":null,"auto_scale":n
                             ull,"fast_anneal":false,"flux_biases":null,"flux_drift_compensation":true,"h_gain_sche
                             dule":null,"initial_state":null,"max_answers":null,"num_reads":100,"programming_therma
                             lization":null,"readout_thermalization":null,"reduce_intersample_correlation":false,"r
                             einitialize_state":null,"embedding_parameters":null,"qpu_backend":"default"},"name":nu
                             ll}

✅ Jobs submitted successfully!
   → Job 1 ID: 68516503feb4512569869b60
   → Job 2 ID: 68516505feb4512569869b62


## 📊 Luna Quantum's Job Management



Luna Quantum's result handling system provides sophisticated job management and analytics capabilities that make working with optimization results intuitive and powerful.

### 🎯 Job Orchestration

#### **Flexible Execution Models:**
- **Synchronous**: `job.result()` blocks until completion - perfect for interactive notebooks
- **Asynchronous**: Non-blocking execution for production pipelines and batch processing

#### **Production-Ready Features:**
- **Automatic retry logic** for transient failures
- **Job persistence** across sessions and system restarts  
- **Resource optimization** with intelligent queuing and scheduling

### 📈 Analytics & Insights

Luna Quantum automatically captures comprehensive execution metrics:
- **Solution quality** assessment and convergence analysis
- **Algorithm performance** benchmarking across different approaches

This enterprise-grade job management ensures reliable, scalable optimization workflows from research to production deployment.

### 📊 Solution Retrieval and Analysis

Luna Quantum provides comprehensive result analysis capabilities:

In [26]:
# Retrieve solutions (this will wait for job completion)
print("⏳ Waiting for job completion...")

tabu_solution = tabu_job.result()
qa_solution = quantum_job.result()

print(f'\nTabu Search found solution with {tabu_solution.best().obj_value}')
print(tabu_solution)

print(f'\nQuantum annealing found solution with {qa_solution.best().obj_value}')
print(tabu_solution)

⏳ Waiting for job completion...


Output()

Output()

2025-06-17 14:52:27 INFO     Sleeping for 5.0 seconds. Waiting and checking a function in a loop.

Output()

Output()

Output()


Tabu Search found solution with 1563.0
x_Ham x_Ham x_Ham x_Mun x_Mun x_Mun x_Bon x_Bon x_Bon │ feas   raw   obj count
    1     0     0     0     0     1     0     1     0 │    t 1.6e3 1.6e3    39
    0     0     1     1     0     0     0     1     0 │    t 1.6e3 1.6e3    61

Total samples: 2
Total variables: 9

Timing:
Total: 10.104441642s

Quantum annealing found solution with 1563.0
x_Ham x_Ham x_Ham x_Mun x_Mun x_Mun x_Bon x_Bon x_Bon │ feas   raw   obj count
    1     0     0     0     0     1     0     1     0 │    t 1.6e3 1.6e3    39
    0     0     1     1     0     0     0     1     0 │    t 1.6e3 1.6e3    61

Total samples: 2
Total variables: 9

Timing:
Total: 10.104441642s


## 🎯 Luna Quantum's Intelligent Solution Processing

![LunaSolve Flow Solution](plots/FlowSolution.png)
Luna Quantum transforms raw optimization results into actionable business insights through sophisticated post-processing capabilities.

### 🧠 Solution Interpretation

#### **Standardized Output Format:**
- **Consistent data structures** across all algorithms and backends
- **Rich metadata** including solution quality, timing, and confidence metrics  
- **Automatic feasibility validation** with constraint satisfaction analysis
- **Multi-solution handling** for exploring alternative optimal solutions

#### **Business Intelligence Integration via Use Case Represenation:**
- **Domain-specific extractors** (tours, schedules, assignments, portfolios)
- **Automated visualization** generation for stakeholder communication
- **Export capabilities** to standard formats (CSV, JSON, databases) for embedding
into existing business systems

### 🚀 From Variables to Decisions

The seamless transformation from optimization variables to real-world decisions showcases Luna Quantum's practical focus. Whether you need delivery routes, resource allocations, or investment portfolios, Luna Quantum bridges the gap between mathematical optimization and business execution.

**Key Advantage**: Luna Quantum's standardized result format means solution processing code works identically whether your optimization ran on classical CPUs, quantum simulators, or real quantum hardware.

### 🗺️ Tour Extraction and Visualization

Transform optimization results into a readable tour format:

In [27]:
# we will continue using the tabu search solution
solution = qa_solution

# Extract tour from solution
tour = extract_tour_from_solution(
    variable_names=solution.variable_names,
    sample_values=solution.best().sample,
    start_city=start_city
)

# Display the optimal tour
print(f"🎯 Optimal Tour:")
print(f"   → Route: {' → '.join(tour)}")

# Calculate total distance
total_distance = 0
for i in range(len(tour) - 1):
    total_distance += distance_matrix[tour[i]][tour[i + 1]]

print(f"   → Total Distance: {total_distance:.1f} km")

🎯 Optimal Tour:
   → Route: Berlin → Hamburg → Bonn → Munich → Berlin
   → Total Distance: 1563.0 km


### 🗺️ Interactive Solution Visualization

Create an interactive map showing the optimal tour:


In [28]:
# Generate interactive map with solution
solution_map = plot_solution_tour(
    G=G,
    tour=tour,
    cities=cities,
    distance_matrix=distance_matrix,
)

print("🗺️  Interactive solution map generated!")
print(f"   → Shows optimal tour: {' → '.join(tour[:-1])}")
print(f"   → Green line indicates optimal route")
print(f"   → Red markers show city locations")

# Display the map
solution_map


🗺️  Interactive solution map generated!
   → Shows optimal tour: Berlin → Hamburg → Bonn → Munich
   → Green line indicates optimal route
   → Red markers show city locations


---

## 🌟 Luna Quantum SDK: Your Gateway to Quantum-Enhanced Optimization

### 🎯 What You've Experienced

This demo showcased Luna Quantum's complete optimization ecosystem through a practical TSP implementation:

✅ **Rapid Development** with predefined optimization patterns  
✅ **Advanced Modeling** using AqModels' flexible framework  
✅ **Multi-Algorithm Execution** comparing classical and quantum approaches  
✅ **Enterprise-Grade Infrastructure** with secure token management  
✅ **Intelligent Result Processing** from variables to business decisions  

### 🚀 Luna Quantum's Competitive Advantages

#### **🔧 Developer Experience**
- **Single API** eliminates the complexity of quantum programming
- **Automatic transformations** handle algorithm-specific requirements
- **Rich documentation** and examples accelerate development
- **Production-ready** from day one with enterprise-grade reliability

#### **⚡ Performance & Scalability**
- **Hybrid classical-quantum** algorithms for optimal performance
- **Cloud-native architecture** scales from prototypes to enterprise workloads
- **Multi-backend support** maximizes hardware utilization

#### **🛡️ Enterprise Ready**
- **Secure token management** for quantum hardware access
- **Role-based access control** for team collaboration
- **Support for hybrid environments** (on-premise + cloud + quantum)

### 🔧 Troubleshooting

**Common Issues and Solutions:**

| Issue | Cause | Solution |
|-------|-------|----------|
| `DWAVE_TOKEN` not found | Environment variable not set | `export DWAVE_TOKEN="your_token"` |
| Quantum job fails | Insufficient credits or token issues | Check D-Wave Leap account, use classical fallback |
| Import errors | Missing dependencies | Run `uv sync` or `pip install -r requirements.txt` |
| Map not displaying | Jupyter notebook rendering | Restart notebook kernel |
| Solution is `None` | Job execution failed | Check error messages, verify token setup |

**Debug Steps:**
1. Verify environment variables: `echo $DWAVE_TOKEN`
2. Test classical algorithms first (no tokens required)
3. Check Luna Quantum logs for detailed error messages
4. Ensure all dependencies are installed correctly

### 🎯 Next Steps & Exploration

**Immediate Extensions:**
- **Add More Cities**: Expand to 6-8 cities for complexity
- **Different Algorithms**: Try Genetic Algorithm, Ant Colony Optimization
- **Parameter Tuning**: Experiment with `num_reads`, annealing schedules

**Advanced Projects:**
- **Multi-Objective TSP**: Minimize time AND cost
- **Vehicle Routing Problem**: Multiple vehicles, capacity constraints
- **Algorithm Analysis**: Compare quantum vs classical performance

**Learning Resources:**
- [Luna Quantum Documentation](https://docs.luna-quantum.com)
- [D-Wave Leap Tutorials](https://cloud.dwavesys.com/leap/)
- [Quantum Computing Fundamentals](https://docs.dwavesys.com/docs/latest/)

### 🚀 Ready to Transform Your Optimization Challenges?

**Start your quantum optimization journey with Luna Quantum SDK today!**

---

*For more information, visit our documentation, explore additional examples, or contact our team for enterprise solutions.*